## Elastic client

In [11]:
!pip install PyPDF2

In [13]:
from elasticsearch import Elasticsearch, helpers
import csv
from PyPDF2 import PdfReader
import time
import pandas as pd
import os

In [14]:
# Credential file
df = pd.read_csv('./elastic credential/watsonx_discovery_credential_tzwx24.csv') # change the path as per your need

In [15]:
# setting up the credential
ELASTIC_USER = df.iloc[0]['username']
ELASTIC_PW = df.iloc[0]['password']
ELASTIC_HOST = df.iloc[0]['watsonx_discovery_url'] # WxD Endpoint
ELASTIC_PORT = df.iloc[0]['port']  #  port number
ELASTIC_CERT_FILE = "./elastic_certificate/es.cert" # path containing certiifcation - change as per your need
INGEST_PIPELINE_NAME="e5-pipeline"
INDEX_NAME_DOC="index-e5"
MODEL_ID=".multilingual-e5-small"
MODEL_ID_E5 = MODEL_ID
INDEX_NAME_E5 = INDEX_NAME_DOC
INGEST_PIPELINE_NAME_E5 = INGEST_PIPELINE_NAME

In [16]:
client = Elasticsearch(
     ELASTIC_HOST+':'+str(ELASTIC_PORT),  # Elasticsearch endpoint
     basic_auth=(ELASTIC_USER, ELASTIC_PW),
     ca_certs=ELASTIC_CERT_FILE,
     verify_certs=True,
     request_timeout=120
)

In [47]:
# This part should be the output of TXT_text_extraction-e5_v3.py
doc_folder = ''
src = f'./data_output_txt2csv_v3/{doc_folder}/{doc_folder}.csv' # change the path as per your need

In [48]:
#Create list of doc objects from csv
docs = []
#with open('pdf_text_2078.csv','r') as csvin:
with open(src,'r') as csvin:
    reader = csv.reader(csvin, delimiter=",")
    for line in reader:
        doc= {
            "_index": INDEX_NAME_E5,
            "pipeline": INGEST_PIPELINE_NAME_E5,
            "_source": {
                        'title': line[0],
                        'url': line[1],
                        'chunk_num': line[2],
                        'text':  line[3]
                        },
            "_id": line[1] + "_" + line[2],    #url+chunk_num
        }
        docs.append(doc)
len(docs)

36

In [49]:
docs

[{'_index': 'index-e5-extended-v2',
  'pipeline': 'e5-pipeline',
  '_source': {'title': '\ufefftitle',
   'url': 'path',
   'chunk_num': 'chunk num',
   'text': 'text'},
  '_id': 'path_chunk num'},
 {'_index': 'index-e5-extended-v2',
  'pipeline': 'e5-pipeline',
  '_source': {'title': 'r44_mm_30_tailrotor',
   'url': 'data_output_txt/r44_mm_30_tailrotor/r44_mm_30_tailrotor.txt',
   'chunk_num': '1',
   'text': 'passage: CHAPTER 30\n\n\nTAIL ROTOR'},
  '_id': 'data_output_txt/r44_mm_30_tailrotor/r44_mm_30_tailrotor.txt_1'},
 {'_index': 'index-e5-extended-v2',
  'pipeline': 'e5-pipeline',
  '_source': {'title': 'r44_mm_30_tailrotor',
   'url': 'data_output_txt/r44_mm_30_tailrotor/r44_mm_30_tailrotor.txt',
   'chunk_num': '2',
   'text': 'passage: | Section Title                                                                                                         | Page                                                                                                                  |\n|-

In [37]:
### Files in CSV format to be ingested

## Insert Documents directly with E5 pipeline

In [50]:
#Use Elasticsearch helper to load docs into index in batches
batch_size = 10
batch = [docs[i:i + batch_size] for i in range(0, len(docs), batch_size)]

for i, chunk in enumerate(batch):
    print(f"Uploading chunk {i+1}/{len(batch)}")
    try:
        helpers.bulk(client, chunk, timeout="360s")
    except Exception as e:
        print(f"Error: {e}. Moving on...")
        time.sleep(10)
        continue

Uploading chunk 1/4
Uploading chunk 2/4
Uploading chunk 3/4
Uploading chunk 4/4
